# Collaborative filtering using WALS algorithm | 1

## Create raw dataset

In [0]:
PROJECT = 'qwiklabs-gcp-4a684069c4776675'
BUCKET = 'colaborative-filtering-agea'
REGION = 'us-central1'

In [0]:
from google.cloud import bigquery
from google.colab import auth

In [0]:
auth.authenticate_user()

In [7]:
client = bigquery.Client(PROJECT)
sql = """
#standardSQL
SELECT 
  user_id as visitorId,
  content_id as contentId,
  COUNT(user_id) as numberOfVisits

  FROM AGEA_ASL.Dataset_B
  GROUP BY contentId,visitorId
"""
df = client.query(sql + 'LIMIT 10000').to_dataframe() # TODO: SACAR LIMIT
df.head()

,visitorId,contentId,numberOfVisits
0,7003253,nLBCbDNAc,1
1,3828642,oadkSDA7v,1
2,4390038,VlhGrp0-M,1
3,6779720,UfolOSfq_,1
4,7438985,0ZpSCTZmI,1


In [0]:
stats = df.describe()

In [9]:
# the rating is the numberOfVisits scaled to be in the range 0-1
median = stats['numberOfVisits']['50%']
df['rating'] = 0.3 * df['numberOfVisits'] / median
df[df['rating'] > 1]['rating'] = 1

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
del df['numberOfVisits']

In [0]:
!rm -rf data
!mkdir data

In [0]:
df.to_csv('data/collab_raw.csv', index=False, header=False)

In [13]:
# upload to bucket just in case that the local csv be erased
!gsutil cp 'data/collab_raw.csv'  gs://{BUCKET}/collab_raw.csv 

Copying file://data/collab_raw.csv [Content-Type=text/csv]...
/ [1 files][215.1 KiB/215.1 KiB]                                                
Operation completed over 1 objects/215.1 KiB.                                    


In [14]:
!head data/collab_raw.csv

7003253,nLBCbDNAc,0.3
3828642,oadkSDA7v,0.3
4390038,VlhGrp0-M,0.3
6779720,UfolOSfq_,0.3
7438985,0ZpSCTZmI,0.3
4614746,mv3bSgwD0,0.3
4894674,mQAcCfHZn,0.6
3729812,-oEBh31DE,0.3
3660364,k4JktPu_S,0.3
6655995,fHJ_wRDU9,0.3
